<a href="https://colab.research.google.com/github/RvsL/kaggle/blob/main/eruptions_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Competition is about prediction of volcano eruption time
https://www.kaggle.com/c/predict-volcanic-eruptions-ingv-oe

  I've already created a model on LGBM with feature engeneering, it worked out 145 place in competition with MAE score around 5,2e6. First place is scored around 3e6, so i need to check different approach. Maybe First place guy knows much about the subject, and i don't. 

  This notebook aims to load data as a pictures and use deep learning to read volcano features right from pictures.

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import f1_score,recall_score,precision_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split


from sklearn.preprocessing import StandardScaler

from keras import regularizers as kreg
from keras.layers import Dense
from keras.models import Sequential

import os
from tqdm import tqdm_notebook
import scipy.fftpack
from scipy.signal import chirp, find_peaks, peak_widths, peak_prominences

import matplotlib.pyplot as plt


In [3]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization

import glob
import os

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
import pickle

hdir = '/content/drive/MyDrive/2 - рабочее/32 - PMI/random/eruptions/'

for var in ['X_val', 'y_val', 'X_train', 'y_train']:
    
    with open(f'{hdir}{var}.pickle', 'rb') as handle:
        globals()[var] = pickle.load(handle)


In [ ]:
from keras import layers
from keras import models


from spektral.layers import GCNConv


from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(600, 600, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1))


# model = models.Sequential()
# model.add(GCNConv(32, activation='elu'))
# model.add(GCNConv(32, activation='elu', kernel_regularizer= kreg.l1(0.01)))
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(1))


model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)